In [ ]:
from pathlib import Path

import Metashape

from mynd.backends import metashape as backend
from mynd.utils.log import logger
from mynd.utils.result import Ok, Err, Result

### Load document into backend

In [ ]:
DOCUMENT_PATH: Path = Path(
    "/data/kingston_snv_01/acfr_revisits_metashape_projects/r23685bc_working_version.psz"
)

result: Result[str, str] = backend.load_project(DOCUMENT_PATH)
match result:
    case Ok(message):
        logger.info(message)
    case Err(message):
        logger.error(message)

### Backend playground

In [ ]:
from mynd.api.camera import StereoCollection
from mynd.api.identifier import Identifier

from mynd.camera import CameraCalibration
from mynd.io.file_database import H5Database, create_file_database, load_file_database

from mynd.geometry import (
    PixelMap,
    RectificationResult,
    compute_stereo_rectification,
    rectify_image_pair,
)

# NOTE: Integrate camera readers and writers into IO library eventually
from mynd.io.camera_io import write_rectification_results_to_file_database


def process_stereo_collection(
    identifier: Identifier, collection: StereoCollection
) -> None:
    """Process a collection of stereo data."""

    rectification: RectificationResult = compute_stereo_rectification(
        collection.calibrations
    )

    logger.info(f"Key:                 {identifier}")
    logger.info(f"Calibrations:        {len(collection.calibrations)}")
    logger.info(f"Image loaders:       {len(collection.image_loaders)}")

    DATABASE_PATH: Path = Path(
        f"/data/kingston_snv_01/test/{identifier.label}_calibration.h5"
    )

    if DATABASE_PATH.exists():
        result: Result[H5Database, str] = load_file_database(DATABASE_PATH)
    else:
        result: Result[H5Database, str] = create_file_database(DATABASE_PATH)

    if result.is_err():
        logger.error(f"Database error: {result}")

    database: H5Database = result.ok()

    result: Result[None, str] = write_rectification_results_to_file_database(
        database,
        group_name="stereo",
        rectification=rectification,
    )

    if result.is_err():
        logger.error(result.err())

    database.get("stereo").visit(lambda item: logger.info(item))


def main():
    """TODO"""
    result: Result = backend.camera_services.request_stereo_data()

    data: dict[Identifier, list[StereoCollection]] = result.unwrap()

    for identifier, collections in data.items():

        for collection in collections:
            process_stereo_collection(identifier, collection)


# ---------- Invoke main function ----------
main()